In [ ]:
import yaml
from pathlib import Path
import requests

In [ ]:
with open("/home/zhouan/App/BenchmarkingClassifiers/snakemake/config/classifiers.yml.template", 'r') as f:
    locations = yaml.load(f, Loader=yaml.FullLoader)
type(locations)
locations["sylph"]["db_path"]

In [ ]:
def fetch_fungus_or_nah(taxids):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/taxonomy/taxon/'
    taxids_str = ','.join(map(str, taxids))
    url = f'{base_url}{taxids_str}'
    params = {}
    try:
        response = s.get(url, params=params)
        response.raise_for_status() # 
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        r = response.json()
    return r
fetch_fungus_or_nah(tmp)

In [ ]:
def fetch_reference_dataset_report(taxon, reference_only='true'):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{taxon}/dataset_report?'
    params = {f'filters.reference_only': {reference_only},
              'filters.exclude_paired_reports': 'false',
              'filters.exclude_atypical': 'true'}
    try:
        response = s.get(base_url, params=params)
        # print(response.url)
        response.raise_for_status()
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return response.json()



In [ ]:
for taxid, seq_abund in theoretical_abundance['species'].items():
    r = fetch_reference_dataset_report(taxid)

r

In [ ]:
len(r['reports'])

In [ ]:
def fetch_reference_sequence_report(accession):
    base_url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/{accession}/sequence_reports?'
    params = {}
    try:
        response = s.get(base_url, params=params)
        response.raise_for_status()
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return response.json()
r = fetch_reference_sequence_report('GCF_000091045.1')
r


In [ ]:
sequences = {}
lengths = 0
for sequence in r['reports']:
    if sequence['assigned_molecule_location_type'] == 'Plasmid':
        pass
    elif sequence['assigned_molecule_location_type'] == 'Chromosome':
        sequences[sequence['refseq_accession']] = sequence['length']
        lengths += sequence['length']
sequences


In [ ]:
lengths

In [ ]:
import json
import logging
import math
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from rich.status import Status
import yaml
from pathlib import Path
theoretical_abundances = {}
data="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers"
for ground_truth_file in ["ground_truth_tax_updated.yml" ,"ground_truth_updated.yml"]:
    ground_truth_file=Path(f"{data}/{ground_truth_file}")
    with open(ground_truth_file, 'r') as f:
        theoretical_abundance = {'species': yaml.load(f, Loader=yaml.FullLoader)}
    theoretical_abundance['genus'] = {}

    for k, v in theoretical_abundance['species'].items():
        genus_name = k.split()[0]
        if genus_name in theoretical_abundance['genus']:
            theoretical_abundance['genus'][genus_name] += v
        else:
            theoretical_abundance['genus'][genus_name] = v

    if ground_truth_file.stem.split('_')[-2] == 'tax':
        theoretical_abundances['tax'] = theoretical_abundance
    else:
        theoretical_abundances['sequence'] = theoretical_abundance
theoretical_abundances

In [ ]:
collected_metrics = {
    'Kraken2': {
        'TP': 10,
        'FP': 2,
        'FN': 3,
        'Precision': 0.833,
        'Recall': 0.769,
        'F1': 0.800
    },
    'MetaPhlAn': {
        'TP': 8,
        'FP': 1,
        'FN': 4,
        'Precision': 0.889,
        'Recall': 0.667,
        'F1': 0.762
    }
}
import pandas as pd

top_table = pd.DataFrame.from_dict(collected_metrics, orient='index').T
print(top_table)


In [ ]:
pd.DataFrame.from_dict(collected_metrics, orient='index')

In [ ]:
groups = ['Classified', 'Classified', 'Unclassified', 'Classified']
list_uc = ['no_hit',
           'ambigious',
           'unclassified',
           'missing_genus',
           'not_distributed',
           'rounding_error'
           ]
import pandas as pd

groups_cat = pd.Categorical(groups, categories=['Classified'] + list_uc)
print(groups_cat)

In [ ]:
groups_cat.categories

In [ ]:
from pathlib import Path
data="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output"
data=Path(data)
data.glob("/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output/species" + '/' + '*.tsv')

In [ ]:
import itertools
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from dataclasses import dataclass
from sklearn import metrics
from typing import List
@dataclass # 通过 @dataclass，可以直接在类中定义属性，而不需要显式编写 __init__ 方法
class ClassificationResults:
    classifier: str
    results: pd.Series
    names_unclassified: List[str]

    def bundled_classified(self, normalized=False): #  将结果分为两类：已分类（Classified）和未分类（list_uc 中的单元）
        """
        'Classified' --> ground truth + classified but not in ground truth
        The rest are in list_uc and are as-is
        """
        groups = np.where(np.isin(self.results.index, list_uc), self.results.index, 'Classified') # 根据 self.results.index 中的分类单元是否在 list_uc 中，将分类单元分为两类：1. 在 list_uc 中的分类单元，保持原样；2. 不在 list_uc 中的分类单元，标记为 'Classified'；返回NumPy 数组
        groups_cat = pd.Categorical(groups,
                                    categories=['Classified'] + list_uc) # 将分类单元分为 'Classified'和 list_uc 中的类
        bundled_classified = self.results.groupby(groups_cat, observed=False).aggregate('sum') # 对'Classified'和 list_uc 的reads数量进行求和
        if normalized:
            return self._normalize_series(bundled_classified)
        return bundled_classified

In [ ]:
list_uc = ['no_hit', 'ambigious', 'unclassified', 'missing_genus']
classifiers_input = [
    ClassificationResults(
        classifier='kraken2',
        results=pd.Series({
            'E_coli': 100,
            'S_aureus': 200,
            'B_subtilis': 50,
            'Unclassified': 30,
            'no_hit': 10,
            'ambigious': 5,
            'unclassified': 2,
            'missing_genus': 1
        }),
        names_unclassified=['no_hit', 'ambigious', 'unclassified', 'missing_genus']
    ),
    ClassificationResults(
        classifier='bracken',
        results=pd.Series({
            'E_coli': 150,
            'S_aureus': 180,
            'B_subtilis': 70,
            'Unclassified': 20,
            'no_hit': 10,
            'ambigious': 5,
            'unclassified': 2,
            'missing_genus': 1
        }),
        names_unclassified=['no_hit', 'ambigious', 'unclassified', 'missing_genus']
    )
]

In [ ]:
un_classified = {}
list_abundance_classifiers = ['MetaPhlAn3', 'mOTUs2']
for classifier_result in classifiers_input:
    if classifier_result.classifier not in list_abundance_classifiers: # 如果分类器不直接输出丰度结果
        un_classified[classifier_result.classifier] = classifier_result.bundled_classified(False) # 返回一个字典，包含Classified'和 list_uc 分类的reads数量之和
un_classified_values = pd.DataFrame.from_dict(un_classified)
output_plots_organism="/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/output/species"
output_plots_organism = Path(output_plots_organism)
un_classified_values.to_csv(output_plots_organism / 'number_table.tsv', sep='\t')

In [ ]:
enumerate(classifiers_input)

MetaPhlAn_test

In [ ]:
import pandas as pd
import re
classification_tax = "/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/TB2001B49C021_00_metagenome.txt"
abundances = []
species_taxids = []
with open(classification_tax, 'r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        # if line.startswith('clade_name'):
        #     header = line.strip().split('\t')
        #     continue
        if re.search(r'\|s__',line):
            clade_name, lineage, abundance = line.strip().split('\t')[:3]
            species_name = int(lineage.strip('|').split('|')[-1])
            species_taxids.append(species_name)
            abundances.append(f'{float(abundance):.5f}')
pd.Series(data=abundances,index=species_taxids).to_csv("/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/test.tsv", sep='\t', header=False)   


In [ ]:
line

In [ ]:
paste <(cut -f 1 {cleaned_abundance} \
        | {TAXONKIT} lineage --data-dir {params.db} \
        | {TAXONKIT} reformat --data-dir {params.db} --format "{{g}}\t{{s}}" --miss-rank-repl "unclassified" \
        | cut -f 3,4 \
        | awk 'BEGIN{{FS="\t"; OFS="\t"}} {{if ($1 == "unclassified" && $2 != "unclassified") {{print "missing_genus", $2}} else {{print}}}}' \
        ) \
        <(cut -f 2 {cleaned_abundance}) \
> {output.cleaned_abundance_tax};

In [ ]:
classification = "/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/TB2001B49C021_00_metagenome.txt"
cleaned_abundance_tax = "/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/classification_cleaned_tax.tsv"
TSV_metaphlan = "/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/output_metaphlan.tsv"
duplicate_names=  '/home/zhouan/Docker_dir/Classification/MetaPhlAn_test/corrected_duplicates_metaphlan.log'

In [ ]:
import pandas as pd

with open(classification, 'r') as f:
    for line in f:
        if line.split('\t')[0] == "UNCLASSIFIED":
            unknown_portion = float(line.strip().split('\t')[:3][-1])

metaphlan_input = pd.read_table(cleaned_abundance_tax,
                                names=['genus', 'species', 'abundance'])

organism_count = pd.Series(metaphlan_input['abundance'].values / 100, index=metaphlan_input["genus"])
organism_count['no_hit'] = unknown_portion / 100

# Sum for the same organisms. This is important with genera as there can be multiple species with the same genus.
organism_count = organism_count.groupby(organism_count.index).sum()

if any(organism_count.index.duplicated(keep=False)):
    organism_count[organism_count.index.duplicated()].to_csv(duplicate_names, sep='\t',index=True,header=False)
    braken2_input=organism_count.groupby(organism_count.index,sort=False).sum()

organism_count.to_csv(TSV_metaphlan, sep='\t', index=True, header=False)
organism_count.index.duplicated()

Kraken2 output

In [ ]:
kraken2_ouput_file = "/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/kraken2/test.txt"
import pandas as pd
kraken2_ouput = pd.read_table(kraken2_ouput_file,
                                header = None,
                                usecols=[0,1,2,3]
                                )
## Remove unclassified maps
kraken2_ouput_clean = kraken2_ouput[kraken2_ouput.iloc[:,0] == 'C']
kraken2_ouput
# kraken2_ouput_file 
"""
C/U:classed or unclassed
Sequence ID
taxonomy ID
序列长度
序列中每个k-mer对应的LCA （空格间隔；taxid ： k-mer numbers）
"""
# kraken2_ouput_clean 文件有前四列

In [ ]:
paste <(cut -f 2,3 {input}) <(cut -f 3 {input} \
        | {TAXONKIT} lineage --data-dir {params.db} \
        | {TAXONKIT} reformat --data-dir {params.db} --format "{{g}}\t{{s}}" --miss-rank-repl "unclassified" \
        | cut -f 3,4 \
        | awk 'BEGIN{{FS="\t"; OFS="\t"}} {{if ($1 == "unclassified" && $2 != "unclassified") {{print "missing_genus", $2}} else {{print}}}}' \
        ) > {output.TSV};

Sylph output

In [38]:
classification_tax = "/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/sylph/classification_tax.tsv"
output_TSV = "/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/sylph/classification_cleaned.tsv"
output_tax_TSV = "/home/zhouan/Docker_dir/Classification/00.extract_method/Vazyme_D6300/BenchmarkingClassifiers/sylph/classification_cleaned_tax.tsv"
import pandas as pd
import re


species_names = []
genus_names = []
abundances = []
relative_abundances = []
sequence_abundances = []
with open(classification_tax, 'r') as f:
    for line in f:
        if line.startswith("#"):
            continue
        if re.search(r'\|s__',line):
            clade_name, relative_abundance, sequence_abundance = line.strip().split('\t')[:3]
            if re.search(r'\|t__',clade_name):
                continue
            else:
                species_name = clade_name.split('|')[-1].split('_')[2]
                genus_name = clade_name.split('|')[-1].split('_')[2].split(' ')[0].split('_')[0]
            species_names.append(species_name)
            genus_names.append(genus_name)
            relative_abundances.append(f'{float(relative_abundance):.5f}')
            sequence_abundances.append(f'{float(sequence_abundance):.5f}')

pd.Series(data=relative_abundances,index=species_names).to_csv(output_TSV, sep='\t', header=False)
cleaned_tax = list(zip(genus_names, species_names, relative_abundances))
pd.DataFrame(cleaned_tax).to_csv(output_tax_TSV, sep='\t', index=False, header=False)


In [39]:
clade_name.split('|')[-1].split('_')

['t', '', 'GCF', '018333375.1']

In [ ]:
# ## 根据species name 找 taxid
# cat species_names.txt | {TAXONKIT} names2taxid --data-dir {params.db} ## 输出species name\ttaxid
# classification_cleaned_tax.tsv # genus\tspecies\tabundance
# data = list(zip(genus_names, species_names, relative_abundances))

# # 将合并后的数据写入到 output_TSV 文件
# df = pd.DataFrame(data, columns=['genus_name', 'species_name', 'relative_abundance'])

# # 将 DataFrame 写入 TSV 文件，使用制表符分隔，且不包含列名（header=False）
# df.to_csv(output_TSV, sep='\t', index=False, header=False)

In [56]:
# ROOT / 'input' / (Path(config['input']['fastq']).stem + '_HQ' + Path(config['input']['fastq']).suffix)
fastq = "/home/zhouan/Cyclone_data/Metagenome/meta_241108/TB2001D03C021/TB2001D03C021.fastq.gz"

from pathlib import Path
p = Path(fastq)
file_name = str(p.stem).split('.')[0]
print(p.stem)   # 输出 'file'
print(file_name) # 输出 '.fastq'
print(p.suffix)    # 输出 'gz'
test = p.stem + '_HQ' + p.suffix
print(test)

TB2001D03C021.fastq
TB2001D03C021
.gz
TB2001D03C021.fastq_HQ.gz


In [47]:
import os

file_path = "/path/to/your/file.fastq.gz"  # 或者 "/path/to/your/file.fq.gz"
file_name = os.path.basename(file_path)  # 提取文件名
file_base_name = os.path.splitext(file_name)[0]  # 去掉扩展名
print(file_base_name)  # 输出 'file'
file_name

file.fastq


'file.fastq.gz'